<a href="https://colab.research.google.com/github/django-frog/Latakian-Accent-ETL-Pipeline/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Latakian Accent ETL Pipeline**

This notebook implements a complete ETL (Extract → Transform → Load) workflow
for building a dialectal Arabic word dataset.

**Stages:**

  1. Environment setup and data path configuration
  2. Audio transcription using OpenAI Whisper
  3. Manual human review (outside notebook)
  4. Text normalization and cleanup
  5. Tokenization using CAMeL Tools
  6. Deduplication (literal + fuzzy)
  7. Preparing for manual enrichment (glossary, POS tagging)

**Drive Access:**

I stored every signle file on my drive account this is the [link](https://drive.google.com/drive/folders/1jc_vwhKyoNUJWpONIsP1ABOzUZm4DkK_?usp=sharing) if you want to discover everything.

Author: Mohammad Hamdan

# Setup Environment

In [1]:
!pip install -q openai-whisper torch soundfile
!sudo apt install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


# Connect with Drive

In [2]:
# Colab cell 1 — mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Define Pathes

In [3]:
DRIVE_AUDIO_DIR = "/content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025"
OUTPUT_CSV = f"{DRIVE_AUDIO_DIR}/latakia_transcriptions_large_acc.csv"

**Check Path Existance**

In [4]:
import os
# List a few audio files
if not os.path.isdir(DRIVE_AUDIO_DIR):
    raise FileNotFoundError(f"Folder not found: {DRIVE_AUDIO_DIR}. Check the path in the cell.")
files = sorted([f for f in os.listdir(DRIVE_AUDIO_DIR) if f.lower().endswith(('.wav','.mp3','.m4a','.flac', 'mpeg'))])
print(f"Found {len(files)} audio files. Example files:\n", files[:20])

Found 20 audio files. Example files:
 ['voice_record_1.mp3', 'voice_record_10.mp3', 'voice_record_11.mp3', 'voice_record_12.mp3', 'voice_record_13.mp3', 'voice_record_14.mp3', 'voice_record_15.mp3', 'voice_record_16.mpeg', 'voice_record_17.mpeg', 'voice_record_18.mpeg', 'voice_record_19.mpeg', 'voice_record_2.mp3', 'voice_record_20.mpeg', 'voice_record_3.mp3', 'voice_record_4.mp3', 'voice_record_5.mp3', 'voice_record_6.mp3', 'voice_record_7.mp3', 'voice_record_8.mp3', 'voice_record_9.mp3']


# Build ETL Pipeline

## Extract Transcript from Voice Records

In the following step, we'll trying to extract the transcript from voice records, so we can get the data to start building our database


### Automation Step

We use **OpenAI Whisper** to automatically transcribe audio recordings into text.

The script:
  - Loads a pretrained model ("**large-v3**" for accuracy)
  - Iterates through all voice samples
  - Generates Arabic transcripts
  - Saves results incrementally to CSV

In [5]:
import whisper
import pandas as pd
import os
from tqdm import tqdm
import time

# Choose model: tiny, base, small, medium, large
MODEL_NAME = "large-v3"   # You can also try "small" or "large-v3" for better accuracy
model = whisper.load_model(MODEL_NAME)
print("✅ Model loaded:", MODEL_NAME)

# Gather audio files
files = sorted([f for f in os.listdir(DRIVE_AUDIO_DIR) if f.lower().endswith(('.wav','.mp3','.m4a','.flac', 'mpeg'))])
print(f"Found {len(files)} audio files")

# Load existing CSV (if any)
if os.path.exists(OUTPUT_CSV):
    df_existing = pd.read_csv(OUTPUT_CSV)
    processed = set(df_existing['filename'].astype(str).tolist())
    print(f"Loaded existing CSV with {len(df_existing)} records; skipping {len(processed)} files.")
else:
    df_existing = pd.DataFrame(columns=["filename","transcription","language","duration","model","error","timestamp"])
    processed = set()

# Buffer for new transcriptions
rows = []

# Main loop
for filename in tqdm(files, desc="Transcribing"):
    if filename in processed:
        print(f"⏩ Skipping already processed: {filename}")
        continue

    file_path = os.path.join(DRIVE_AUDIO_DIR, filename)
    ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    try:
        result = model.transcribe(file_path, language="ar")
        text = result.get("text", "").strip()
        duration = result.get("duration", None)
        row = {
            "filename": filename,
            "transcription": text,
            "language": result.get("language", "ar"),
            "duration": duration,
            "model": MODEL_NAME,
            "error": "",
            "timestamp": ts
        }
        print(f"[OK] {filename} -> {len(text.split())} words")
    except Exception as e:
        print(f"[ERR] {filename} -> {e}")
        row = {
            "filename": filename,
            "transcription": "",
            "language": "",
            "duration": None,
            "model": MODEL_NAME,
            "error": str(e),
            "timestamp": ts
        }

    # Add to list of new rows
    rows.append(row)

# ✅ Save all results *once* at the end
if rows:
    df_new = pd.DataFrame(rows)
    df = pd.concat([df_existing, df_new], ignore_index=True)
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ Saved {len(df)} total records to {OUTPUT_CSV}")
else:
    print("⚠️ No new files were processed.")


✅ Model loaded: large-v3
Found 20 audio files


Transcribing:   5%|▌         | 1/20 [00:13<04:17, 13.53s/it]

[OK] voice_record_1.mp3 -> 56 words


Transcribing:  10%|█         | 2/20 [00:29<04:31, 15.09s/it]

[OK] voice_record_10.mp3 -> 91 words


...

[OK] voice_record_7.mp3 -> 73 words


Transcribing:  95%|█████████▌| 19/20 [05:47<00:15, 15.62s/it]

[OK] voice_record_8.mp3 -> 46 words


Transcribing: 100%|██████████| 20/20 [05:55<00:00, 17.79s/it]

[OK] voice_record_9.mp3 -> 50 words
✅ Saved 20 total records to /content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/latakia_transcriptions_large_acc.csv


### Review Step

This Step is so necessary, we need to review the results of the AI model, before moving to the next step, I will do this step manually and then use the cleaned transcript into the next step.

**I will store the final reviewed transcripts into separate path, so next steps will depend on it**




## Transcript Transformation Stage

**Goal**: Clean and normalize the reviewed transcript

  - Remove noise, diacritics, elongation, and non-Arabic symbols

  - Produce a uniform Arabic text suitable for tokenization

### Normalization

Some normalization rules will be applied on the transcript, before start any transformation process

In [4]:
DRIVE_AUDIO_DIR = "/content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025"
REVIEW_TRANSCRIPT = f"{DRIVE_AUDIO_DIR}/reviewed_transcript/reviewed_latakia_transcriptions_large_acc.csv"
NORMALIZED_PATH = f"{DRIVE_AUDIO_DIR}/reviewed_transcript/normalized_latakia_transcriptions.csv"
TOKENIZED_TRANSCRIPT = f"{DRIVE_AUDIO_DIR}/reviewed_transcript/tokenized_latakia_transcriptions.csv"
DEDUPED_PATH = f"{DRIVE_AUDIO_DIR}/reviewed_transcript/unique_latakia_tokens.csv"


In [5]:
# ======================================================
#  Arabic Text Normalization for Reviewed Transcripts
#  Simplified & production-grade (no hamza or variant mapping)
# ======================================================

import re
import unicodedata
import pandas as pd



# Precompiled regexes
RE_DIACRITICS = re.compile(r'[\u0610-\u061A\u064B-\u065F\u06D6-\u06ED]')
RE_NON_ARABIC = re.compile(r'[^ء-ي0-9\u0660-\u0669A-Za-z\s\-]')
RE_MULTI_SPACE = re.compile(r'\s+')
RE_REPEAT = re.compile(r'(.)\1{2,}')  # 3 or more repeated chars → collapse

# ------------------------
# Utility functions
# ------------------------
def unicode_nfc(text):
    """Normalize text to canonical Unicode form."""
    return unicodedata.normalize('NFC', text)

def remove_bom_and_trim(text):
    """Remove BOM and trim whitespace."""
    return text.replace('\ufeff', '').replace('\u200f', '').replace('\u200e','').strip()

def remove_invisible(text):
    """Remove invisible formatting characters (tatweel, zero-width joiners)."""
    text = text.replace('\u0640', '')  # tatweel
    text = text.replace('\u200c', '')  # zero width non-joiner
    text = text.replace('\u200d', '')  # zero width joiner
    return text

def remove_diacritics(text):
    """Remove Arabic tashkeel (harakat)."""
    return RE_DIACRITICS.sub('', text)


def remove_non_arabic_punct(text):
    """Remove punctuation and symbols except Arabic/Latin letters, digits, hyphen, and spaces."""
    return RE_NON_ARABIC.sub(' ', text)

def collapse_repeats(text, max_repeats=2):
    """Collapse elongated letters like حلوووو → حلو."""
    def repl(m):
        ch = m.group(1)
        return ch * max_repeats
    return RE_REPEAT.sub(repl, text)

def final_cleanup(text):
    """Collapse extra spaces, lower Latin, trim."""
    text = RE_MULTI_SPACE.sub(' ', text).strip()
    text = text.lower()  # lowercase Latin only
    return text

# ------------------------
# Master normalization function
# ------------------------
def normalize_text(text):
    """Apply normalization pipeline to one string, return normalized text + steps log."""
    if not isinstance(text, str):
        return text, []

    steps = []
    s = unicode_nfc(text); steps.append("unicode_nfc")
    s = remove_bom_and_trim(s); steps.append("trim_bom")
    s = remove_invisible(s); steps.append("remove_invisible")
    s = remove_diacritics(s); steps.append("remove_diacritics")
    s = remove_non_arabic_punct(s); steps.append("remove_punct")
    s = collapse_repeats(s, max_repeats=2); steps.append("collapse_repeats")
    s = final_cleanup(s); steps.append("final_cleanup")

    return s, steps

# ------------------------
# Apply to your dataset
# ------------------------


# Load the dataset
df = pd.read_csv(REVIEW_TRANSCRIPT, encoding="utf-8")

# Ensure we have a column named 'transcription'
if "transcription" not in df.columns:
    raise ValueError("The CSV must contain a column named 'transcription'.")

# Apply normalization
normalized_texts = []
steps_logs = []

for text in df["transcription"]:
    norm, steps = normalize_text(text)
    normalized_texts.append(norm)
    steps_logs.append("|".join(steps))

df["normalized_transcription"] = normalized_texts
df["normalization_steps"] = steps_logs

# Save the result
df.to_csv(NORMALIZED_PATH, index=False, encoding="utf-8-sig")

print(f"✅ Normalization complete. Saved to:\n{NORMALIZED_PATH}")
print(f"Total rows processed: {len(df)}")

# Display sample
df.head(10)


✅ Normalization complete. Saved to:
/content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/reviewed_transcript/normalized_latakia_transcriptions.csv
Total rows processed: 19


,filename,transcription,language,duration,model,error,timestamp,normalized_transcription,normalization_steps
0,voice_record_1.mp3,اخويا ابو العبد على راسي حارتك والله عين اخويا...,ar,NaN,large-v3,NaN,2025-10-12 19:44:34,اخويا ابو العبد على راسي حارتك والله عين اخويا...,unicode_nfc|trim_bom|remove_invisible|remove_d...
1,voice_record_2.mp3,بأديش بدك تحسبلنا ياه لا كثير مغلي علينا جارك ...,ar,NaN,large-v3,NaN,2025-10-12 19:44:42,بأديش بدك تحسبلنا ياه لا كثير مغلي علينا جارك ...,unicode_nfc|trim_bom|remove_invisible|remove_d...
2,voice_record_3.mp3,يعني والله عادية الله يساعد مثلا هون منقوبة ما...,ar,NaN,large-v3,NaN,2025-10-12 19:45:07,يعني والله عادية الله يساعد مثلا هون منقوبة ما...,unicode_nfc|trim_bom|remove_invisible|remove_d...
3,voice_record_4.mp3,والله عادية ولو ماما منثلة السوق معبية عادية م...,ar,NaN,large-v3,NaN,2025-10-12 19:45:12,والله عادية ولو ماما منثلة السوق معبية عادية م...,unicode_nfc|trim_bom|remove_invisible|remove_d...
4,voice_record_5.mp3,امشي ماما امشي عمو ما بيحبنا لكي ضبا خلاص عم ب...,ar,NaN,large-v3,NaN,2025-10-12 19:45:47,امشي ماما امشي عمو ما بيحبنا لكي ضبا خلاص عم ب...,unicode_nfc|trim_bom|remove_invisible|remove_d...
5,voice_record_6.mp3,لكي ضبطلنا ياها يلعو قلبك نفقت قلبي صار لي من ...,ar,NaN,large-v3,NaN,2025-10-12 19:45:52,لكي ضبطلنا ياها يلعو قلبك نفقت قلبي صار لي من ...,unicode_nfc|trim_bom|remove_invisible|remove_d...
6,voice_record_7.mp3,ما رَن برن برن شيبك ليش هيك بصلتك محروقة الا ...,ar,NaN,large-v3,NaN,2025-10-15 19:24:39,ما رن برن برن شيبك ليش هيك بصلتك محروقة الا ما...,unicode_nfc|trim_bom|remove_invisible|remove_d...
7,voice_record_8.mp3,شو هالزكاة أنسة ديبة لكن شو رأيك نسألو نقلو اذ...,ar,NaN,large-v3,NaN,2025-10-15 19:25:17,شو هالزكاة أنسة ديبة لكن شو رأيك نسألو نقلو اذ...,unicode_nfc|trim_bom|remove_invisible|remove_d...
8,voice_record_9.mp3,آسفين أنسة ديبة غلطنا بحقك مونيك غشيمة هاتشوف ...,ar,NaN,large-v3,NaN,2025-10-15 19:25:23,آسفين أنسة ديبة غلطنا بحقك مونيك غشيمة هاتشوف ...,unicode_nfc|trim_bom|remove_invisible|remove_d...
9,voice_record_10.mp3,شو يعني هلأ ؟ نتركه بلا أكل و بدون شراب ؟ بديع...,ar,NaN,large-v3,NaN,2025-10-15 19:22:46,شو يعني هلأ نتركه بلا أكل و بدون شراب بديعة شب...,unicode_nfc|trim_bom|remove_invisible|remove_d...


### Tokenization

After normalizing the transcripts, we need to tokenize them, so we can have a meaningful units to store them inside the database.

To make it easier, we'll use a liberary for CAML Tools

In [6]:
!pip install camel-tools tqdm pandas --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_vers

In [7]:
from camel_tools.tokenizers.word import simple_word_tokenize
from tqdm import tqdm
import pandas as pd
import re
import os


# ------------------------------------------------------
# Load data
# ------------------------------------------------------
df = pd.read_csv(NORMALIZED_PATH, encoding="utf-8-sig")
if "normalized_transcription" not in df.columns:
    raise ValueError("Missing column 'normalized_transcription' in your CSV.")

print(f"Loaded {len(df)} records from {NORMALIZED_PATH}")

# ------------------------------------------------------
# Tokenization utilities
# ------------------------------------------------------

def basic_token_clean(token):
    """Clean token from stray punctuation/spaces."""
    token = re.sub(r"[^\u0600-\u06FF0-9A-Za-z]", "", token)  # keep Arabic & alphanum
    token = token.strip()
    return token

def tokenize_text(text):
    """Tokenize Arabic text using CAMeL Tools."""
    tokens = simple_word_tokenize(str(text))
    tokens = [basic_token_clean(tok) for tok in tokens if tok.strip()]
    return tokens

# ------------------------------------------------------
# Tokenize all transcripts
# ------------------------------------------------------
all_tokens = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Tokenizing transcripts"):
    filename = row.get("filename", f"row_{i}")
    text = row["normalized_transcription"]

    tokens = tokenize_text(text)
    for token in tokens:
        if token:  # skip empty
            all_tokens.append({
                "filename": filename,
                "token": token,
                "transcription_id": i
            })

# ------------------------------------------------------
# Save tokenized output
# ------------------------------------------------------
tokens_df = pd.DataFrame(all_tokens)
tokens_df.to_csv(TOKENIZED_TRANSCRIPT, index=False, encoding="utf-8-sig")

print(f"✅ Tokenization complete.")
print(f"Total tokens: {len(tokens_df)}")
print(f"Output saved to: {TOKENIZED_TRANSCRIPT}")

# Preview few results
tokens_df.head(20)

Loaded 19 records from /content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/reviewed_transcript/normalized_latakia_transcriptions.csv


Tokenizing transcripts: 100%|██████████| 19/19 [00:00<00:00, 40.44it/s]


✅ Tokenization complete.
Total tokens: 1212
Output saved to: /content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/reviewed_transcript/tokenized_latakia_transcriptions.csv


,filename,token,transcription_id
0,voice_record_1.mp3,اخويا,0
1,voice_record_1.mp3,ابو,0
2,voice_record_1.mp3,العبد,0
3,voice_record_1.mp3,على,0
4,voice_record_1.mp3,راسي,0
5,voice_record_1.mp3,حارتك,0
6,voice_record_1.mp3,والله,0
7,voice_record_1.mp3,عين,0
8,voice_record_1.mp3,اخويا,0
9,voice_record_1.mp3,لما,0


#### Remove Literal Duplicates

In [8]:
# ======================================================
#  Deduplicate Tokens → Unique Words Only
# ======================================================

import pandas as pd
import re


# Load token data
df = pd.read_csv(TOKENIZED_TRANSCRIPT, encoding="utf-8-sig")
print(f"Loaded {len(df)} tokens from {TOKENIZED_TRANSCRIPT}")

# ------------------------------------------------------
# Basic cleanup before deduplication
# ------------------------------------------------------
def normalize_token(token):
    """Clean token to unify duplicates."""
    token = str(token).strip()
    token = re.sub(r"[^\u0600-\u06FF0-9A-Za-z]", "", token)  # Keep Arabic & alphanum only
    token = re.sub(r"\s+", "", token)
    return token

df["clean_token"] = df["token"].apply(normalize_token)
df = df[df["clean_token"].astype(bool)]  # Drop empty tokens

# ------------------------------------------------------
# Deduplicate
# ------------------------------------------------------
unique_tokens = df["clean_token"].drop_duplicates().reset_index(drop=True)
unique_df = pd.DataFrame({"id": range(1, len(unique_tokens) + 1), "token": unique_tokens})
unique_df['filename'] = df['filename']

# ------------------------------------------------------
# Save output
# ------------------------------------------------------
unique_df.to_csv(DEDUPED_PATH, index=False, encoding="utf-8-sig")

print(f"✅ Unique token list generated.")
print(f"Total unique tokens: {len(unique_df)}")
print(f"Saved to: {DEDUPED_PATH}")

# Preview
unique_df.head(20)


Loaded 1212 tokens from /content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/reviewed_transcript/tokenized_latakia_transcriptions.csv
✅ Unique token list generated.
Total unique tokens: 717
Saved to: /content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/reviewed_transcript/unique_latakia_tokens.csv


,id,token,filename
0,1,اخويا,voice_record_1.mp3
1,2,ابو,voice_record_1.mp3
2,3,العبد,voice_record_1.mp3
3,4,على,voice_record_1.mp3
4,5,راسي,voice_record_1.mp3
5,6,حارتك,voice_record_1.mp3
6,7,والله,voice_record_1.mp3
7,8,عين,voice_record_1.mp3
8,9,لما,voice_record_1.mp3
9,10,لمطلك,voice_record_1.mp3


#### Remove Similar Words

This time we'll do a fuzzy comparison between words, so we can discover more duplicated words


In [9]:
!pip install rapidfuzz pandas tqdm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.7 MB/s eta 0:00:00


In [10]:
# ======================================================
#   Detect Near-Duplicate Arabic Tokens (Fuzzy Matching)
#   + Include line numbers for human review
# ======================================================

import pandas as pd
from rapidfuzz import fuzz
from tqdm import tqdm

# ------------------------------------------------------
# Paths
# ------------------------------------------------------
DUPLICATES_PATH = f"{DRIVE_AUDIO_DIR}/reviewed_transcript/possible_duplicates_with_lines.csv"

# ------------------------------------------------------
# Load tokens
# ------------------------------------------------------
df = pd.read_csv(DEDUPED_PATH, encoding="utf-8-sig")
if "token" not in df.columns:
    raise ValueError("The file must contain a 'token' column.")

tokens = df["token"].dropna().tolist()
print(f"Loaded {len(tokens)} tokens for fuzzy comparison.")

# ------------------------------------------------------
# Settings
# ------------------------------------------------------
SIMILARITY_THRESHOLD = 78  # 85-90 is typical for Arabic dialect variants (78% for precision)

# ------------------------------------------------------
# Compare tokens pairwise
# ------------------------------------------------------
checked_pairs = set()
duplicates = []

for i, token in tqdm(enumerate(tokens), total=len(tokens), desc="Comparing tokens"):
    for j in range(i + 1, len(tokens)):
        other = tokens[j]
        pair = tuple(sorted([token, other]))
        if pair in checked_pairs:
            continue
        score = fuzz.token_sort_ratio(token, other)
        if score >= SIMILARITY_THRESHOLD and token != other:
            duplicates.append({
                "line_1": i + 2,   # +2 because pandas adds header row (index starts from 0)
                "token_1": token,
                "line_2": j + 2,
                "token_2": other,
                "similarity": score
            })
        checked_pairs.add(pair)

# ------------------------------------------------------
# Save results
# ------------------------------------------------------
if duplicates:
    dup_df = pd.DataFrame(duplicates).sort_values("similarity", ascending=False)
    dup_df.to_csv(DUPLICATES_PATH, index=False, encoding="utf-8-sig")
    print(f"✅ Found {len(dup_df)} potential duplicates.")
    print(f"Saved to: {DUPLICATES_PATH}")
else:
    print("✅ No fuzzy duplicates detected above threshold.")

# Preview top results
if duplicates:
    dup_df.head(20)


Loaded 717 tokens for fuzzy comparison.


Comparing tokens: 100%|██████████| 717/717 [00:00<00:00, 1709.14it/s]


✅ Found 252 potential duplicates.
Saved to: /content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/reviewed_transcript/possible_duplicates_with_lines.csv


**Note:** For More Flexibility, The duplicated words are stored in a separate file, so users can make sure it's really presenting the same meaning.

## Complete Information

Most of information should be filled by human like glossary Arabic for each word, because ther's no AI model that can predict or give the meaning for these words, So I have took a sample and fill it by my own.

In [12]:
DATA_SAMPLE_PATH = f"{DRIVE_AUDIO_DIR}/final_sample.csv"

In [13]:
!pip install stanza pandas --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.4 MB/s eta 0:00:00


### Word Position

for each word, I found the position, I've used **Stanza**, which is an AI model that can detect the position instead of manual input.

In [14]:
import stanza
import pandas as pd

# --------------------------------------------------
# Download the Arabic model (run once)
# --------------------------------------------------
stanza.download("ar")

# --------------------------------------------------
# Initialize the Arabic NLP pipeline
# --------------------------------------------------
nlp = stanza.Pipeline(lang="ar", processors="tokenize,pos", use_gpu=True)

# --------------------------------------------------
# Load your dataset
# --------------------------------------------------
df = pd.read_csv(DATA_SAMPLE_PATH, encoding="utf-8-sig")

# --------------------------------------------------
# POS tagging helper function
# --------------------------------------------------
def get_pos_stanza(text):
    """Return the main POS tag for the input Arabic text using Stanza."""
    if not isinstance(text, str) or not text.strip():
        return ""
    doc = nlp(text)
    pos_tags = [word.upos for sent in doc.sentences for word in sent.words]
    if not pos_tags:
        return "UNK"
    # Return the most frequent POS tag
    return max(set(pos_tags), key=pos_tags.count)

# --------------------------------------------------
# Apply POS tagging to your gloss_ar column
# --------------------------------------------------
df["pos"] = df["gloss_ar"].apply(get_pos_stanza)

# --------------------------------------------------
# Save the result
# --------------------------------------------------
OUTPUT_PATH = DATA_SAMPLE_PATH.replace(".csv", "_with_pos_stanza.csv")
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

print(f"✅ POS tagging done! Saved to:\n{OUTPUT_PATH}")
df.head()


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ar (Arabic) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ar/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ar (Arabic):
| Processor | Package     |
---------------------------
| tokenize  | padt        |
| mwt       | padt        |
| pos       | padt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


✅ POS tagging done! Saved to:
/content/drive/MyDrive/latakian_accent_samples/voice_samples_12_oct_2025/final_sample_with_pos_stanza.csv


,id,dialect_region,token,gloss_ar,source,filename,source_url,pos
0,1,Latakian,اخويا,أخي,voice_file,voice_record_1.mp3,NaN,X
1,2,Latakian,راسي,راسي,voice_file,voice_record_1.mp3,NaN,X
2,3,Latakian,حارتك,الحي,voice_file,voice_record_1.mp3,NaN,NOUN
3,4,Latakian,والله,أقسم بالله,voice_file,voice_record_1.mp3,NaN,VERB
4,5,Latakian,عيّن,أنظر,voice_file,voice_record_1.mp3,NaN,VERB


**Measuring the model effeciency**

for missing pos types, we can fill them manually

In [15]:

total_rows = len(df)
invalid_rows = df["pos"].isin(["X", "UNK"]).sum()
valid_rows = total_rows - invalid_rows

efficiency = (valid_rows / total_rows) * 100

print(f"🔍 Total rows: {total_rows}")
print(f"❌ Invalid tags (X or UNK): {invalid_rows}")
print(f"✅ Valid tags: {valid_rows}")
print(f"📊 Efficiency: {efficiency:.2f}%")

🔍 Total rows: 355
❌ Invalid tags (X or UNK): 91
✅ Valid tags: 264
📊 Efficiency: 74.37%


## Load Dataset to PostGreSQL

In this step, we'll integrate with **supabase** so we can use **PostGreSQL**, And upload our dataset.

In [16]:
!pip install supabase pandas python-dotenv --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:00


### Renaming and Prepare Dataset

Rename columns so it makes scense, fill the NaN values with an empty string, so we can upload them correcty to the database.

In [17]:
DATA_PATH = f"{DRIVE_AUDIO_DIR}/final_sample_with_pos_stanza.csv"
df = pd.read_csv(DATA_PATH, encoding="utf-8-sig")

# Select and rename columns to match your table
df_to_upload = df.rename(columns={
    "token": "word_or_phrase",
    "gloss_ar": "gloss_ar",
    "pos": "pos",
    "dialect_region": "dialect_region",
    "source": "source",
    "filename": "filename",
    "source_url": "source_url",
})[[
    "dialect_region", "word_or_phrase", "gloss_ar",
    "pos", "source", "filename", "source_url"
]]

# Replace NaN values (important for JSON encoding)
df_to_upload = df_to_upload.where(pd.notnull(df_to_upload), None)

print("✅ Data prepared for upload.")
df_to_upload.head()

✅ Data prepared for upload.


,dialect_region,word_or_phrase,gloss_ar,pos,source,filename,source_url
0,Latakian,اخويا,أخي,X,voice_file,voice_record_1.mp3,None
1,Latakian,راسي,راسي,X,voice_file,voice_record_1.mp3,None
2,Latakian,حارتك,الحي,NOUN,voice_file,voice_record_1.mp3,None
3,Latakian,والله,أقسم بالله,VERB,voice_file,voice_record_1.mp3,None
4,Latakian,عيّن,أنظر,VERB,voice_file,voice_record_1.mp3,None


### Setup Supabase Connection

In [18]:
import os
from supabase import create_client, Client
from google.colab import userdata


SUPABASE_URL = userdata.get('task_4_client_url')
SUPABASE_KEY = userdata.get('task_4_access_token')


supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print("✅ Connected to Supabase.")


✅ Connected to Supabase.


### Insert Data into Supabase

In [19]:
batch_size = 100
for start in range(0, len(df_to_upload), batch_size):
    end = start + batch_size
    batch = df_to_upload.iloc[start:end].to_dict(orient="records")
    response = supabase.table("latakian_words").insert(batch).execute()
    print(f"✅ Uploaded rows {start}–{end}: {len(response.data) if response.data else 'OK'}")

✅ Uploaded rows 0–100: 100
✅ Uploaded rows 100–200: 100
✅ Uploaded rows 200–300: 100
✅ Uploaded rows 300–400: 55


### Verify data is inserted successfully

In [20]:
response = supabase.table("latakian_words").select("*").limit(5).execute()
for row in response.data:
    print(row)


{'id': 387, 'created_at': '2025-10-23T20:29:01.028414+00:00', 'dialect_region': 'Latakian', 'word_or_phrase': 'بيلبسا', 'gloss_ar': 'يرتديها', 'transliteration': None, 'pos': 'X', 'source': 'voice_file', 'filename': 'voice_record_4.mp3', 'source_url': None, 'notes': None}
{'id': 388, 'created_at': '2025-10-23T20:29:01.028414+00:00', 'dialect_region': 'Latakian', 'word_or_phrase': 'ضبا', 'gloss_ar': 'وظبها', 'transliteration': None, 'pos': 'VERB', 'source': 'voice_file', 'filename': 'voice_record_5.mp3', 'source_url': None, 'notes': None}
{'id': 389, 'created_at': '2025-10-23T20:29:01.028414+00:00', 'dialect_region': 'Latakian', 'word_or_phrase': 'مبيقيان', 'gloss_ar': 'لم يبقى', 'transliteration': None, 'pos': 'PART', 'source': 'voice_file', 'filename': 'voice_record_5.mp3', 'source_url': None, 'notes': None}
{'id': 390, 'created_at': '2025-10-23T20:29:01.028414+00:00', 'dialect_region': 'Latakian', 'word_or_phrase': 'غير', 'gloss_ar': 'إلا', 'transliteration': None, 'pos': 'PART', 'so